<a href="https://colab.research.google.com/github/numair07/NLQtoSPARQL/blob/main/Question_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
file = open("/content/train-data.json")
train_data = json.load(file)
file = open("/content/test-data.json")
test_data = json.load(file)

In [ ]:
for dt in train_data:
    print(dt['corrected_question'])
    print(dt['sparql_query'])
    print()

Streaming output truncated to the last 5000 lines.
ASK WHERE { <http://dbpedia.org/resource/Mathura–Vadodara_Section> <http://dbpedia.org/ontology/routeEnd> <http://dbpedia.org/resource/Vadodara_Junction_railway_station> }

Stan Van Gundy serves as a head coach and president for which basketball team?
 SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/property/president> <http://dbpedia.org/resource/Stan_Van_Gundy> } 

Which appliance's CPU is Cell (microprocessor) and predecessor is PlayStation 2?
SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/ontology/cpu> <http://dbpedia.org/resource/Cell_(microprocessor)> . ?uri <http://dbpedia.org/ontology/predecessor> <http://dbpedia.org/resource/PlayStation_2>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/InformationAppliance>}

People who debuted from Houston Astros have played in which clubs?
SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/ontology/debutTeam> <http://dbpedia.org/resource/Ho

In [ ]:
boolean = 0
list_ques = 0
count = 0
total = 0
for dt in train_data:
    if("ASK" in dt['sparql_query']):
        boolean += 1
    elif("COUNT" in dt['sparql_query']):
        count += 1
    elif("SELECT" in dt['sparql_query']):
        list_ques += 1
    total += 1

####Train Data

In [ ]:
print("Total : ", total)
print("Boolean : ", boolean)
print("Count : ", count)
print("List : ", list_ques)

Total :  4000
Boolean :  285
Count :  535
List :  3180


In [ ]:
boolean = 0
list_ques = 0
count = 0
total = 0
for dt in test_data:
    if("ASK" in dt['sparql_query']):
        boolean += 1
    elif("COUNT" in dt['sparql_query']):
        count += 1
    elif("SELECT" in dt['sparql_query']):
        list_ques += 1
    total += 1

####Test Data

In [ ]:
print("Total : ", total)
print("Boolean : ", boolean)
print("Count : ", count)
print("List : ", list_ques)

Total :  1000
Boolean :  83
Count :  123
List :  794


In [ ]:
import pandas as pd
ques = []
type_ = []
#ASK is type 3, COUNT is type 2, SELECT is type 1
for dt in train_data:
    ques.append(str(dt['corrected_question']))
    if("ASK" in dt['sparql_query']):
        type_.append(3)
    elif("COUNT" in dt['sparql_query']):
        type_.append(2)
    elif("SELECT" in dt['sparql_query']):
        type_.append(1)

train = pd.DataFrame({"question" : ques, "type" : type_})

ques = []
type_ = []
for dt in test_data:
    ques.append(str(dt['corrected_question']))
    if("ASK" in dt['sparql_query']):
        type_.append(3)
    elif("COUNT" in dt['sparql_query']):
        type_.append(2)
    elif("SELECT" in dt['sparql_query']):
        type_.append(1)

test = pd.DataFrame({"question" : ques, "type" : type_})

In [ ]:
train.head(5)

,question,type
0,How many movies did Stanley Kubrick direct?,2
1,Which city's foundeer is John Forbes?,1
2,What is the river whose mouth is in deadsea?,1
3,What is the allegiance of John Kotelawala ?,1
4,How many races have the horses bred by Jacques...,2


In [ ]:
test.head(5)

,question,type
0,Which architect of Marine Corps Air Station Ka...,1
1,Which sects people are part of local political...,1
2,List common systems are developed by of the Go...,1
3,For which games are Sam Loyd and Eric Schiller...,1
4,Is Peter Piper Pizza in the pizza industry?,3


In [ ]:
test[test['type']==3]

,question,type
4,Is Peter Piper Pizza in the pizza industry?,3
14,Is William H Blanchard buried in Colorado?,3
32,is blue lagoon island a part of the bahamas ac...,3
34,Did the New Way party merge in the One Israel ...,3
40,Is Seattle the Ship builder of USS Marmora (IX...,3
...,...,...
964,Is the Vadodara railway junction onwed by Indi...,3
970,Is Latin the official language of Sovereign Mi...,3
974,Does Ford own the Ford Kansas Assembly plant?,3
982,Did Stanlee write the Tales of Suspense?,3


In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
x_train = train['question']
y_train = train['type']
x_test = test['question']
y_test = test['type']

In [ ]:
def clean_sentences(text):
    text = text.lower()
    text = " ".join(text.split())
    text = " ".join(lemmatizer.lemmatize(word) for word in text.split())
    return text

In [ ]:
x_train = x_train.map(lambda a:clean_sentences(a))
x_test = x_test.map(lambda a:clean_sentences(a))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.08, max_df=0.50,smooth_idf=False)
vectorizer.fit(x_train)
x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions) * 100)

92.4


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       0.95      0.96      0.95       794
           2       0.93      0.73      0.82       123
           3       0.70      0.90      0.79        83

    accuracy                           0.92      1000
   macro avg       0.86      0.86      0.85      1000
weighted avg       0.93      0.92      0.92      1000



In [ ]:
#predict quesion
question = input("Enter your question : ")
question = clean_sentences(question)
question = vectorizer.transform([question])
prediction = model.predict(question)
print(prediction)

Enter your question : Is India a country?
[3]
